In [25]:
import torch
torch.cuda.empty_cache()

In [30]:


import io
import os
import torch
import warnings
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [2]:
set_seed(123)

# Number of training epochs (authors on fine-tuning Bert recommend between 2 and 4).
epochs = 4

# Number of batches - depending on the max sequence length and GPU memory.
# For 512 sequence length batch of 10 works without cuda memory issues.
# For small sequence length can try batch of 32 or higher.
batch_size = 32

# Pad or truncate text sequences to a specific length
# if `None` it will use maximum sequence of word piece tokens allowed by model.
max_length = 60

# Look for gpu to use. Will use `cpu` by default if no gpu found.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model_name_or_path='skt/kogpt2-base-v2'

# Dictionary of labels and their id - this will be used to convert.
# String labels to number ids.
labels_ids = {'sad':0,'sadness':0, 'anger':1,'angry':1,'disgust':2,'happiness':3,'fear':4,
              'neutral':5,'surprise':6}

# How many labels are we using in training.
# This is used to decide size of classification head.
n_labels = len(labels_ids)



In [3]:
import os

dir_path = "./dataset"
data_list = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.csv' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_csv(file_path,encoding="cp949"))
        if '.xlsx' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_excel(file_path))

In [43]:
data = data_list[0].iloc[:,1:3]

for _ in range(1,3):
    data = pd.concat([data,data_list[_].iloc[:,1:3]])
buf = data_list[3].iloc[:,1:]
#buf.columns = ['발화문',"a상황"]
#data = pd.concat([data,buf])
data.columns = ["context","label"]

In [45]:
data

,context,label
0,"어, 청소 니가 대신 해 줘!",anger
1,둘 다 청소 하기 싫어. 귀찮아.,anger
2,둘 다 하기 싫어서 화내.,anger
3,그럼 방세는 어떡해.,anger
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad
...,...,...
19369,나 엘리베이터에 갇혔어.,fear
19370,하지만 기분이 나쁜 걸 어떡해?,angry
19371,자취방 엘리베이턴데 정전인가봐.,fear
19372,나 드디어 프로젝트 끝났어!,happiness


In [44]:
data[data["label"]=="sadness"]["label"] ="sad"
data[data["label"]=="anger"]["label"]="angry"

In [29]:
data.label.value_counts().index

Index(['sad', 'angry', 'disgust', 'happiness', 'fear', 'neutral', 'surprise'], dtype='object', name='label')

In [6]:

#encoder = LabelEncoder()
#data["label"] = encoder.fit_transform(data["label"])

In [46]:
train_x, val_x, train_y, val_y = train_test_split(data["context"],data["label"],test_size = 0.2,stratify=data["label"])

In [47]:
train_data = pd.DataFrame(columns=["context","label"])
train_data["context"]=train_x
train_data["label"]=train_y

In [48]:
train_data

,context,label
9981,아직 안 해봤는데 해보는 게 좋을까?,anger
18541,완전 놀랐지 진짜. 다들 소리지르고 난리 났어.,surprise
193,다행히 생명에 지장은 없대.,anger
14886,나 엘리베이터에 갇혔어.,fear
375,회사가 바쁘다는 핑계로 과장님이 나한테만 일을 몰아주는 느낌이야.,anger
...,...,...
17889,그것도 좋은 생각이네.,happiness
11650,응. 다리만 조금 다치셨어.,anger
8884,갑자기 해피가 막 달려와서 주변 사람들이 놀라긴 했어.,surprise
7710,다행히 사람이 얼마 없었어.,surprise


In [38]:
data

,context,label
0,angry,angry
1,angry,angry
2,angry,angry
3,angry,angry
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad
...,...,...
19369,나 엘리베이터에 갇혔어.,fear
19370,하지만 기분이 나쁜 걸 어떡해?,angry
19371,자취방 엘리베이턴데 정전인가봐.,fear
19372,나 드디어 프로젝트 끝났어!,happiness


In [8]:
class NewsDataset(Dataset):

    def __init__(self, data , use_tokenizer):

    # Check if path exists.
        """if not os.path.isdir(path):
          # Raise error if path is invalid.
          raise ValueError('Invalid `path` variable! Needs to be a directory')"""
        """self.texts = []
        self.labels = []

        # Since the labels are defined by folders with data we loop 
        # through each label.
        for label in ['ITscience', 'culture','economy','entertainment','health','life','politic','social','sport']:
          sentiment_path = os.path.join(path, label)

          # Get all files from path.
          files_names = os.listdir(sentiment_path)#[:10] # Sample for debugging.
          # Go through each file and read its content.
            for file_name in tqdm(files_names, desc=f'{label} files'):
                file_path = os.path.join(sentiment_path, file_name)

                # Read content.
                content = io.open(file_path, mode='r', encoding='utf-8').read()
                # Fix any unicode issues.
                content = fix_text(content)
                # Save content.
                self.texts.append(content)
                # Save encode labels.
                self.labels.append(label)
        """
        self.texts = np.array(data["context"])
        self.labels =np.array(data["label"])
        # Number of exmaples.
        self.n_examples = len(self.labels)


        return

    def __len__(self):
        r"""When used `len` return the number of examples.

        """
    
        return self.n_examples

    def __getitem__(self, item):
        r"""Given an index return an example from the position.

        Arguments:

          item (:obj:`int`):
              Index position to pick an example to return.

        Returns:
          :obj:`Dict[str, str]`: Dictionary of inputs that contain text and 
          asociated labels.

        """

        return {'text':self.texts[item],
                'label':self.labels[item]}



class Gpt2ClassificationCollator(object):
    r"""
    Data Collator used for GPT2 in a classificaiton rask. 
    
    It uses a given tokenizer and label encoder to convert any text and labels to numbers that 
    can go straight into a GPT2 model.

    This class is built with reusability in mind: it can be used as is as long
    as the `dataloader` outputs a batch in dictionary format that can be passed 
    straight into the model - `model(**batch)`.

    Arguments:

      use_tokenizer (:obj:`transformers.tokenization_?`):
          Transformer type tokenizer used to process raw text into numbers.

      labels_ids (:obj:`dict`):
          Dictionary to encode any labels names into numbers. Keys map to 
          labels names and Values map to number associated to those labels.

      max_sequence_len (:obj:`int`, `optional`)
          Value to indicate the maximum desired sequence to truncate or pad text
          sequences. If no value is passed it will used maximum sequence size
          supported by the tokenizer and model.

    """

    def __init__(self, use_tokenizer, labels_encoder, max_sequence_len=None):

        # Tokenizer to be used inside the class.
        self.use_tokenizer = use_tokenizer
        # Check max sequence length.
        self.max_sequence_len = use_tokenizer.model_max_length if max_sequence_len is None else max_sequence_len
        # Label encoder used inside the class.
        self.labels_encoder = labels_encoder

        return

    def __call__(self, sequences):
        r"""
        This function allowes the class objesct to be used as a function call.
        Sine the PyTorch DataLoader needs a collator function, I can use this 
        class as a function.

        Arguments:

          item (:obj:`list`):
              List of texts and labels.

        Returns:
          :obj:`Dict[str, object]`: Dictionary of inputs that feed into the model.
          It holddes the statement `model(**Returned Dictionary)`.
        """
    
        # Get all texts from sequences list.
        texts = [sequence['text'] for sequence in sequences]
        # Get all labels from sequences list.
        labels = [sequence['label'] for sequence in sequences]
        # Encode all labels using label encoder.
        labels = [self.labels_encoder[label] for label in labels]
        # Call tokenizer on all texts to convert into tensors of numbers with 
        # appropriate padding.
        inputs = self.use_tokenizer(text=texts, return_tensors="pt", padding=True, truncation=True,  max_length=self.max_sequence_len)
        # Update the inputs with the associated encoded labels as tensor.
        inputs.update({'labels':torch.tensor(labels)})

        return inputs







In [9]:
def train(dataloader, optimizer_, scheduler_, device_):
    """
      Train pytorch model on a single pass through the data loader.

      It will use the global variable `model` which is the transformer model 
      loaded on `_device` that we want to train on.

      This function is built with reusability in mind: it can be used as is as long
        as the `dataloader` outputs a batch in dictionary format that can be passed 
        straight into the model - `model(**batch)`.

      Arguments:

          dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
              Parsed data into batches of tensors.

          optimizer_ (:obj:`transformers.optimization.AdamW`):
              Optimizer used for training.

          scheduler_ (:obj:`torch.optim.lr_scheduler.LambdaLR`):
              PyTorch scheduler.

          device_ (:obj:`torch.device`):
              Device used to load tensors before feeding to model.

      Returns:

          :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
            Labels, Train Average Loss].
    """

          # Use global variable for model.
    global model

          # Tracking variables.
    predictions_labels = []
    true_labels = []
          # Total loss for this epoch.
    total_loss = 0

          # Put the model into training mode.
    model.train()

      # For each batch of training data...
    for batch in tqdm(dataloader, total=len(dataloader)):

            # Add original labels - use later for evaluation.
        true_labels += batch['labels'].numpy().flatten().tolist()

            # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

            # Always clear any previously calculated gradients before performing a
            # backward pass.
        model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this a bert model function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to calculate training accuracy.
        loss, logits = outputs[:2]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
        total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
        loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
        optimizer_.step()

            # Update the learning rate.
        scheduler_.step()

            # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()

            # Convert these logits to list of predicted labels values.
        predictions_labels += logits.argmax(axis=-1).flatten().tolist()

      # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

      # Return all true labels and prediction for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss


In [10]:
def validation(dataloader, device_):
    """Validation function to evaluate model performance on a 
      separate set of data.

      This function will return the true and predicted labels so we can use later
      to evaluate the model's performance.

      This function is built with reusability in mind: it can be used as is as long
        as the `dataloader` outputs a batch in dictionary format that can be passed 
        straight into the model - `model(**batch)`.

      Arguments:

        dataloader (:obj:`torch.utils.data.dataloader.DataLoader`):
              Parsed data into batches of tensors.

        device_ (:obj:`torch.device`):
              Device used to load tensors before feeding to model.

      Returns:

        :obj:`List[List[int], List[int], float]`: List of [True Labels, Predicted
            Labels, Train Average Loss]
    """

  # Use global variable for model.
    global model

  # Tracking variables
    predictions_labels = []
    true_labels = []
      #total loss for this epoch.
    total_loss = 0

  # Put the model in evaluation mode--the dropout layers behave differently
  # during evaluation.
    model.eval()

  # Evaluate data for one epoch
    for batch in tqdm(dataloader, total=len(dataloader)):

        # add original labels
        true_labels += batch['labels'].numpy().flatten().tolist()

        # move batch to device
        batch = {k:v.type(torch.long).to(device_) for k,v in batch.items()}

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(**batch)

            # The call to `model` always returns a tuple, so we need to pull the 
            # loss value out of the tuple along with the logits. We will use logits
            # later to to calculate training accuracy.
            loss, logits = outputs[:2]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_loss += loss.item()

            # get predicitons to list
            predict_content = logits.argmax(axis=-1).flatten().tolist()

            # update list
            predictions_labels += predict_content

  # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

  # Return all true labels and prediciton for future evaluations.
    return true_labels, predictions_labels, avg_epoch_loss
     

In [11]:
# Get model configuration.
print('Loading configuraiton...')
model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path=model_name_or_path, num_labels=n_labels)

# Get model's tokenizer.
print('Loading tokenizer...')
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token


# Get the actual model.
print('Loading model...')
model = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path, config=model_config)

# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

# Load model to defined device.
model.to(device)
print('Model loaded to `%s`'%device)

Loading configuraiton...
Loading tokenizer...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Loading model...


Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded to `cuda`


In [12]:
# Create data collator to encode text and labels into numbers.
gpt2_classificaiton_collator = Gpt2ClassificationCollator(use_tokenizer=tokenizer, 
                                                          labels_encoder=labels_ids, 
                                                          max_sequence_len=max_length)


print('Dealing with Train...')
# Create pytorch dataset.
train_dataset = NewsDataset(data =data, 
                               use_tokenizer=tokenizer)
print('Created `train_dataset` with %d examples!'%len(train_dataset))

# Move pytorch dataset into dataloader.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=gpt2_classificaiton_collator)
print('Created `train_dataloader` with %d batches!'%len(train_dataloader))

print()

#print('Dealing with Validation...')
# Create pytorch dataset.
#valid_dataset = NewsDataset(path='/content/drive/MyDrive/newsTopicClassification/sampling_test_data', 
#                               use_tokenizer=tokenizer)
#print('Created `valid_dataset` with %d examples!'%len(valid_dataset))

# Move pytorch dataset into dataloader.
#valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=gpt2_classificaiton_collator)
#print('Created `eval_dataloader` with %d batches!'%len(valid_dataloader))

Dealing with Train...
Created `train_dataset` with 43991 examples!
Created `train_dataloader` with 1375 batches!



In [19]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # default is 1e-8.
                  )

# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Store the average loss after each epoch so we can plot them.
all_loss = {'train_loss':[], 'val_loss':[]}
all_acc = {'train_acc':[], 'val_acc':[]}

# Loop through each epoch.
print('Epoch')
for epoch in tqdm(range(epochs)):
    print()
    print('Training on batches...')
    # Perform one full pass over the training set.
    train_labels, train_predict, train_loss = train(train_dataloader, optimizer, scheduler, device)
    train_acc = accuracy_score(train_labels, train_predict)

  # Get prediction form model on validation data. 
    print('Validation on batches...')
    #valid_labels, valid_predict, val_loss = validation(valid_dataloader, device)
    #val_acc = accuracy_score(valid_labels, valid_predict)

  # Print loss and accuracy values to see how training evolves.
    #print("  train_loss: %.5f - val_loss: %.5f - train_acc: %.5f - valid_acc: %.5f"%(train_loss, val_loss, train_acc, val_acc))
    print("train_loss: {0:.5f} - train_acc: {1:.5f}".format(train_loss, train_acc))
    print()

  # Store the loss value for plotting the learning curve.
    all_loss['train_loss'].append(train_loss)
    #all_loss['val_loss'].append(val_loss)
    all_acc['train_acc'].append(train_acc)
    #all_acc['val_acc'].append(val_acc)



Epoch


  0%|          | 0/4 [00:00<?, ?it/s]


Training on batches...


  0%|          | 0/1375 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Plot loss curves.
plot_dict(all_loss, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

# Plot accuracy curves.
plot_dict(all_acc, use_xlabel='Epochs', use_ylabel='Value', use_linestyles=['-', '--'])

In [ ]:
!conda list

In [14]:
 torch.cuda.is_available() 

True

In [15]:
torch.cuda.get_device_name(0) 

'NVIDIA GeForce RTX 4060 Laptop GPU'

In [16]:
print(device)

cuda


In [23]:

print('Device:', device)
print('가능',torch.cuda.device_count())
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())
print(torch.cuda.get_device_name())

Device: cuda
가능 1
Current cuda device: 0
Count of using GPUs: 1
NVIDIA GeForce RTX 4060 Laptop GPU


In [18]:
print(torch.rand(5,4,device = 'cuda'))

tensor([[0.7958, 0.9474, 0.1798, 0.3609],
        [0.4148, 0.9100, 0.4440, 0.7035],
        [0.2047, 0.0620, 0.7587, 0.7663],
        [0.5932, 0.4696, 0.1425, 0.8497],
        [0.4470, 0.5743, 0.6445, 0.0902]], device='cuda:0')
